SCREENSHOTS: 
In the order of S, B, B', S'

![](images/first.png)
![](images/second.png)
![](images/fourth.png)

![](images/third.png)

In [1]:
numeric_tolerance = 1e-8

In [2]:
# meshfile, label_file = 'data/hand.off', 'data/hand.label.npy'
meshfile, label_file = 'data/woody-hi.off', 'data/woody-hi.label.npy'
# meshfile, label_file = 'data/cactus.off', 'data/cactus.label.npy'
# meshfile, label_file = 'data/cylinder.off', 'data/cylinder.label.npy'

# Given

## Setup

In [3]:
import numpy as np
import igl
import meshplot as mp
from scipy.spatial.transform import Rotation
import ipywidgets as iw
import time
import scipy.sparse as sp


In [4]:
u, f = igl.read_triangle_mesh(meshfile)
labels = np.load(label_file).astype(int)
u -= u.min(axis=0)
u /= u.max()

Error: bad line (0)


In [5]:
handle_vertex_positions = u.copy()
pos_f_saver = np.zeros((labels.max() + 1, 6))
def pos_f(s,x,y,z, α, β, γ):
    slices = (labels==s)
    r = Rotation.from_euler('xyz', [α, β, γ], degrees=True)
    v_slice = u[slices] + np.array([[x,y,z]])
    center = v_slice.mean(axis=0)
    handle_vertex_positions[slices] = r.apply(v_slice - center) + center
    pos_f_saver[s - 1] = [x,y,z,α,β,γ]
    t0 = time.time()
    v_deformed = pos_f.deformer(handle_vertex_positions)
    p.update_object(vertices = v_deformed)
    t1 = time.time()
    print('FPS', 1/(t1 - t0))
pos_f.deformer = lambda x:x

In [6]:
def widgets_wrapper():
    segment_widget = iw.Dropdown(options=np.arange(labels.max()) + 1)
    translate_widget = {i:iw.FloatSlider(min=-1, max=1, value=0) 
                        for i in 'xyz'}
    rotate_widget = {a:iw.FloatSlider(min=-90, max=90, value=0, step=1) 
                     for a in 'αβγ'}

    def update_seg(*args):
        (translate_widget['x'].value,translate_widget['y'].value,
        translate_widget['z'].value,
        rotate_widget['α'].value,rotate_widget['β'].value,
        rotate_widget['γ'].value) = pos_f_saver[segment_widget.value]
    segment_widget.observe(update_seg, 'value')
    widgets_dict = dict(s=segment_widget)
    widgets_dict.update(translate_widget)
    widgets_dict.update(rotate_widget)
    return widgets_dict

In [7]:
def position_deformer(target_pos):
    '''Fill in this function to change positions'''
    return target_pos
''' (Optional) Register this function to perform interactive deformation
pos_f.deformer = position_deformer
'''

' (Optional) Register this function to perform interactive deformation\npos_f.deformer = position_deformer\n'

## Output

In [8]:
# ## Widget UI

p = mp.plot(handle_vertex_positions, f, c=labels)
iw.interact(pos_f,
            **widgets_wrapper())

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0833333…

interactive(children=(Dropdown(description='s', options=(1, 2), value=1), FloatSlider(value=0.0, description='…

<function __main__.pos_f(s, x, y, z, α, β, γ)>

# Step 1:  Removal of high-frequency details

## get A matrix

In [9]:
# vec \subset U = {0, 1, ..., |universe|-1}
# returns U \ vec
def set_inversion(vec, universe):
    return np.array(sorted(set(range(universe)) - set(vec)))

In [10]:
def A_matrix(original_mesh_vertices,f): 
    # L_w * M^-1 * L_w
    L_w = igl.cotmatrix(original_mesh_vertices, f)
    M = igl.massmatrix(original_mesh_vertices, f, igl.MASSMATRIX_TYPE_VORONOI)
    M_inv = sp.linalg.inv(M)
    A = L_w @ M_inv @ L_w
    return A

In [11]:
# A block matrices diagonally and zeros elsewhere
def A_p_marix(original_mesh_vertices, f): 
    # get A and set up for the larger matrix 
    A = A_matrix(original_mesh_vertices, f)
    rows, cols = A.shape
    bigger_matrix = sp.csc_matrix((3 * rows, 3 * cols), dtype=A.dtype)
    
    # Copy matrix A into the appropriate positions of the bigger matrix
    bigger_matrix[:rows, :cols] = A                # Top left
    bigger_matrix[rows:2*rows, cols:2*cols] = A    # Middle middle
    bigger_matrix[2*rows:, 2*cols:] = A            # Bottom right
    return bigger_matrix

In [12]:
# cidx is the constrained indices
def get_A_ff_A_fc(A, cidx, total_indices): 
    # also get non constrained faces
    ncidx = set_inversion(cidx, total_indices)
    
    # A_ff, A_fc
    A_ff = A[ncidx, :][:, ncidx]
    A_fc = A[ncidx, :][:, cidx]
    return A_ff, A_fc

## Precomputing `A`

Since we have that the handles are always the same vertices, we assume `cidx` is always the same. This also means we can safely have `A_ff` and `A_fc` be constant throughout.

In [13]:
def get_u_p_c_cf(original_mesh_vertices, segments): 
    pass

In [14]:
# This function is meant to be called only once. 
def precompute_A_ff_A_fc(original_mesh_vertices, f, segments):
    # note that original_mesh_vertices is just u. 
    _, c_p_idx = get_u_p_c_cf(original_mesh_vertices, segments)
    A_p = A_p_marix(original_mesh_vertices, f)
    len_of_u = len(original_mesh_vertices) * 3
    A_ff, A_fc = get_A_ff_A_fc(A_p, c_p_idx, total_indices=len_of_u)
    return A_ff, A_fc

## get the u_c vector, and the `cf` vector

In [15]:
# vec is a vector of 3D points
# transforming to `p` means stacking the 3D points into a single vector in the form [vx1,...,vxn,vy1,...,vyn,vz1,...,vzn]
def transform_vec_to_p(vec): 
    return np.hstack((vec[:, 0], vec[:, 1], vec[:, 2]))

In [16]:
# get the constrained for per-vertex
# stacked x, y, z
def get_u_p_c_cf(v, segments): 
    # get the constrained values as points, then stack them
    u_c = v[segments != 0]
    u_p_c = transform_vec_to_p(u_c)
    
    #  get the indices of the constrained points, then get tthe indices of the stacked versions [cidx, 1 * len(v) + cidx, 2 * len(v) + cidx]
    c_idx = np.where(segments != 0)[0]
    c_p_idx = np.hstack((c_idx, len(v) + c_idx, 2 * len(v) + c_idx))
    
    return u_p_c, c_p_idx

## Solve the linear system to get u_f and u 

In [17]:
# this function just does the solver. A_ff and A_fc is precomputed. 
# Note to self that an earlier git push version is doing the same thing without precomputed A. 
def get_u_f(u_p_c, A_ff, A_fc): 
    RHS = - A_fc @ u_p_c
    u_f = sp.linalg.spsolve(A_ff, RHS)
    return u_f

In [18]:
# map from "correct order index" : "correct order value" for the constrained values
def get_cidx_to_c_map(cidx, c): 
    mp = {}
    for i in range(cidx.shape[0]): 
        mp[cidx[i]] = c[i]
    return mp

In [19]:
def get_ordered_u(u_f, u_c, cidx): 
    u = np.zeros(u_f.shape[0] + u_c.shape[0])
    cidx_set = set(cidx)
    u_n = u_f.shape[0] + u_c.shape[0]
    mp = get_cidx_to_c_map(cidx, u_c)
    for i in range(u_n): 
        # if the current index is in the set of constrained faces, then we need to get the value from u_c
        if i in cidx_set:
            u[i] = mp[i]
        else:  
            u[i] = u_f[0]
            u_f = u_f[1:]
    return u

In [20]:
def get_u(v, segments, A_ff, A_fc): 
    # first get the stacked version of `u`
    u_p_c, c_p_idx = get_u_p_c_cf(v, segments)
    u_f_p = get_u_f(u_p_c, A_ff, A_fc)
    u_p = get_ordered_u(u_f_p, u_p_c, c_p_idx)
    
    # then reshape it back to the original shape
    n = len(v)
    u = np.vstack((u_p[:n], u_p[n:2*n], u_p[2*n:])).T
    return u 

## Required Output

In [21]:
A_ff, A_fc = precompute_A_ff_A_fc(u, f, labels)
B = get_u(u, labels, A_ff, A_fc)
mp.plot(B, f, shading={"wireframe": True})

/opt/homebrew/lib/python3.10/site-packages/scipy/sparse/_index.py:137: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray_sparse(i, j, x)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0833333…

# Step 2:  Deforming the smooth mesh


You do the same things, except the new vertices passed in are the handle_vertices, which are just changed values

In [22]:
# A_ff, A_fc = precompute_A_ff_A_fc(u, f, labels)
B_p = get_u(handle_vertex_positions, labels, A_ff, A_fc)
mp.plot(B_p, f, shading={'wireframe': True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0833333…

# Step 3: Transferring high-frequency details to the deformed surface

## For each vertex. Vector reference frames and coefficients in `B` and `B'`

In [23]:
# expressing a vector in terms of basis vectors - you can verfy this method using the column way of matrix multiplication
def basis_representation(x, b1, b2, b3): 
    # Construct the matrix B with the basis vectors as columns
    # Solve the equation x = Bc for the coefficients c
    B = np.column_stack((b1, b2, b3))
    c = np.linalg.solve(B, x)
    assert(((b1 * c[0] + b2 * c[1] + b3 * c[2] - x) < numeric_tolerance).all())
    return c 

# n is the normal, e is the edge, and we project e onto the tangent plane of n
# this is from Gram-Schmidt
def projection_onto_tangent(n, e):
    return e - (np.dot(e,n)/np.dot(n,n)) * n

def vector_normalize(v): 
    return v / np.linalg.norm(v)

In [24]:
# for every vertex in B, we want it's three basis vectors, and also the edge we used so that we can use it in B'
def get_vertex_basis_B(vertex_pos, vertex_normal, adjacent_vertices_pos): 
    # 1. The unit vertex normal
    n = vertex_normal
    
    # 2.1. v's outgoing edges
    edges = adjacent_vertices_pos - vertex_pos
    # 2.2. projection of one of v's outgoing edges onto the tangent plane
    max_projection, original_edge_index = None, -1
    for i in range(edges.shape[0]): 
        # edge whose projection onto the tangent plane is longest
        e = edges[i]
        projection = projection_onto_tangent(n, e)
        if max_projection is None or np.linalg.norm(projection) > np.linalg.norm(max_projection):
            max_projection, original_edge_index = projection, i
    # 2.3 normalized
    p = vector_normalize(max_projection)
    
    # 3. The cross-product between (1) and (2)
    k = np.cross(n, p)
    
    # return the basis vectors and the edge
    assert(np.dot(n, p) < numeric_tolerance and np.dot(n, k) < numeric_tolerance and np.dot(p, k) < numeric_tolerance)
    return n, p, k, original_edge_index

In [25]:
# for B` => this is completely different from the previous one
# this is easier beccasue step 2 is partly done for you
def get_vertex_basis_B_p(vertex_normal, projection_edge): 
    # 1. The unit vertex normal
    n = vertex_normal
    
    # 2. projection of `projection_edge` onto the tangent plane
    p_unormalized = projection_onto_tangent(n, projection_edge)
    p = vector_normalize(p_unormalized)
    
    # 3. The cross-product between (1) and (2)
    k = np.cross(n, p)
    
    # return the basis vectors and the edge
    assert(np.dot(n, p) < numeric_tolerance and np.dot(n, k) < numeric_tolerance and np.dot(p, k) < numeric_tolerance)
    return n, p, k

## Get vertex positions based on high frequency detail for all vertices

In [26]:
def precompute_additional_B_info(B_verts, f):
    l = B_verts.shape[0]
    vertex_normals = igl.per_vertex_normals(B_verts, f, igl.PER_VERTEX_NORMALS_WEIGHTING_TYPE_UNIFORM)
    adj = igl.adjacency_list(f)
    n_vec, p_vec, k_vec, original_edge_index_vec = np.zeros((B_verts.shape[0], 3)), np.zeros((B_verts.shape[0], 3)), np.zeros((B_verts.shape[0], 3)), np.zeros(B_verts.shape[0], dtype=int)
    for i in range(l): 
        n_vec[i], p_vec[i], k_vec[i], original_edge_index_vec[i] = get_vertex_basis_B(vertex_pos=B_verts[i], 
                                                                                      vertex_normal=vertex_normals[i], 
                                                                                      adjacent_vertices_pos=B_verts[adj[i]])
    adj = igl.adjacency_list(f)
    B_additional_info = (n_vec, p_vec, k_vec, original_edge_index_vec, adj)
    return B_additional_info

In [27]:
# get the new vertex position of u'
# for each vertex u in B and it's corresponding vertex u' in B'
# note we have the original v which are the vertices of the mesh
# the best way to remember or understand what is happening is imagining visually. 
def get_v_p_new_pos(v, u, u_p, f, B_additional_info): 
    assert(u.shape == u_p.shape == v.shape)
    n_vec, p_vec, k_vec, original_edge_index_vec, adj = B_additional_info
    vertex_prime_normals = igl.per_vertex_normals(u_p, f, igl.PER_VERTEX_NORMALS_WEIGHTING_TYPE_UNIFORM)
    l = u.shape[0]
    v_p = np.zeros(u_p.shape)
    
    for i in range(l): 
        # basis of `B`
        n, p, k, original_edge_index = n_vec[i], p_vec[i], k_vec[i], original_edge_index_vec[i]
        
        # deformed vertex to it's original position
        # expressed as coefficients of the basis vectors
        u_to_v = v[i] - u[i]
        coefficients = basis_representation(u_to_v, n, p, k)
        
        # basis of `B'`
        selected_edge_dst = u_p[adj[i]][original_edge_index] # we want to use the same edge as we did with `u`, but with the actual deformed edge in B'
        selected_edge = selected_edge_dst - u_p[i]
        n_p, p_p, k_p = get_vertex_basis_B_p(vertex_normal=vertex_prime_normals[i], 
                                             projection_edge=selected_edge)
                                             
        # using the same coefficients in the primed basis
        u_p_to_v_p = coefficients[0] * n_p + coefficients[1] * p_p + coefficients[2] * k_p
        v_p[i] = u_p[i] + u_p_to_v_p
        
    return v_p

## Required Output

In [28]:
# not needed precomputation -- and data about B
# A_ff, A_fc = precompute_A_ff_A_fc(u, f, labels)
# B = get_u(u, labels, A_ff, A_fc)
B_additional_info = precompute_additional_B_info(B, f)

# the `u` and `B` can get confusing. Essentially, in `u`, we have the original mesh vertices, and in `B`, we have the deformed mesh vertices.
B_p = get_u(handle_vertex_positions, labels, A_ff, A_fc)
v_p = get_v_p_new_pos(v=u, u=B, u_p=B_p, f=f, B_additional_info=B_additional_info)
mp.plot(v_p, f)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0833333…

# Outputs

In [29]:
2

2

In [30]:
# ## Widget UI

p = mp.plot(handle_vertex_positions, f, c=labels)
iw.interact(pos_f,
            **widgets_wrapper())

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0833333…

interactive(children=(Dropdown(description='s', options=(1, 2), value=1), FloatSlider(value=0.0, description='…

<function __main__.pos_f(s, x, y, z, α, β, γ)>

In [31]:
A_ff, A_fc = precompute_A_ff_A_fc(u, f, labels)
B = get_u(u, labels, A_ff, A_fc)
B_p = get_u(handle_vertex_positions, labels, A_ff, A_fc)
B_additional_info = precompute_additional_B_info(B, f)
B_p = get_u(handle_vertex_positions, labels, A_ff, A_fc)
v_p = get_v_p_new_pos(v=u, u=B, u_p=B_p, f=f, B_additional_info=B_additional_info)

In [32]:
p = mp.subplot(u, f, shading={"wireframe": True}, s=[4, 1, 0])
mp.subplot(B, f, shading={"wireframe": True}, s=[4, 1, 1], data=p)
mp.plot(B_p, f, shading={'wireframe': True}, s=[4, 1, 2], data=p)
mp.plot(v_p, f, s=[4, 1, 3], data=p, shading={"wireframe": True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.316666…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.316669…

# Optional Task: Performance

## Note to grader

I was unable to install sckit-sparse, so I was only able to use .splu. I have brought this up with TA Arvi, and he said it was fine to use .splu. The real thing with sckit-sparse will make it a little faster, so it would have likely made my code faster and work within the required time limits.


In [ ]:
from scipy.sparse.linalg import splu

In [ ]:
# We speed up by using Cholesky decomposition on A_ff
# this function just does the solver. A_ff and A_fc is precomputed. 
# Note to self that an earlier git push version is doing the same thing without precomputed A. 
def get_u_f(u_p_c, A_ff, A_fc): 
    # A_ff is now a spLU object
    RHS = - A_fc @ u_p_c
    u_f = A_ff.solve(RHS)
    return u_f

In [ ]:
A_ff_splu = splu(A_ff)
B_additional_info = precompute_additional_B_info(B, f)

In [ ]:
def position_deformer(target_pos):
    '''Fill in this function to change positions'''
    B_p = get_u(target_pos, labels, A_ff_splu, A_fc)
    v_p = get_v_p_new_pos(u, B, B_p, f, B_additional_info)
    return v_p
''' (Optional) Register this function to perform interactive deformation
pos_f.deformer = position_deformer
'''

## To see the output, watch the video attached

In [ ]:
p = mp.plot(handle_vertex_positions, f, c=labels, shading={"wireframe": True})
iw.interact(pos_f,
            **widgets_wrapper())
pos_f.deformer = position_deformer